In [1]:
import sklearn 
print(sklearn.__version__)

0.21.3


In [2]:
# Load data
import pandas as pd
import numpy as np
#Save the data and notebook in the same folder! 
data=pd.read_csv('transaction.csv')
 
# identify categorical and continous features 
cat=['account_id','type','operation','bank']
ordi=['date','k_symbol']
conti=['amount','balance']

In [3]:
# split data into training and testing set
from sklearn.model_selection import train_test_split
train, test = train_test_split(data,test_size=0.3) 

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
column_trans = ColumnTransformer(
[('ohe', OneHotEncoder(dtype='int',handle_unknown='ignore'),cat),
 ('ord',OrdinalEncoder() , ordi)],
remainder='passthrough')

In [5]:
column_trans.fit_transform(train)

<161325x3484 sparse matrix of type '<class 'numpy.float64'>'
	with 1253562 stored elements in Compressed Sparse Row format>

In [12]:
#following code will raise value error, since testing set has unknown value!
column_trans.transform(test)

In [16]:
from sklearn.preprocessing._encoders import _BaseEncoder
class new_OrdinalEncoder(_BaseEncoder):
    def __init__(self,cat_index='all'):
        self.dicts={}
        # cate_index is the categorical feature index list
        self.cat_index=cat_index
     
    def fit(self,df,*y):
        if self.cat_index=='all':
            self.cat_index=list(range(df.shape[1]))
        for feat in self.cat_index:
            dic=np.unique(df.iloc[:,feat])
            dic=dict([(i,index) for index, i in enumerate(dic)])
            self.dicts[feat]=dic
             
    def fit_transform(self,df,*y):
        if self.cat_index=='all':
            self.cat_index=list(range(df.shape[1]))
        df_output=df.copy()
        for feat in self.cat_index:
            dic=np.unique(df.iloc[:,feat])
            dic=dict([(i,index) for index, i in enumerate(dic)])
            self.dicts[feat]=dic
            df_output.iloc[:,feat]=df.iloc[:,feat].apply(lambda x: dic[x])
        return df_output
         
    def transform(self,df):
        df_output=df.copy()
        for feat in self.cat_index:
            dic=self.dicts[feat]
            df_output.iloc[:,feat]=df.iloc[:,feat].apply(self.unknown_value,args=(dic,))
        return df_output
     
    def unknown_value(self,value,dic): # It will set up a new interger for unknown values!
        try:
            return dic[value]
        except:
            return len(dic)

In [17]:
#if using my new_ordinalencoder, it will not raise error anymore!
column_trans = ColumnTransformer(
[('ohe', OneHotEncoder(dtype='int',handle_unknown='ignore'),cat),
 ('ord',new_OrdinalEncoder() , ordi)],
remainder='passthrough')

In [18]:
column_trans.fit_transform(train)

<161325x3484 sparse matrix of type '<class 'numpy.float64'>'
	with 1253562 stored elements in Compressed Sparse Row format>

In [19]:
column_trans.transform(test)

<69140x3484 sparse matrix of type '<class 'numpy.float64'>'
	with 537339 stored elements in Compressed Sparse Row format>